## Model Predictive Control Implementation

In [1]:
import numpy as np
from qpsolvers import solve_qp

## Transform the ss model to a ss model with embedded integrator

In order to include an embedded integrator in our MPC approach we need to redefine the variables we are going to use as the variation of the inputs, states, and next states:
$$
\begin{align*}
\Delta x(k+1) &= x(k+1) - x(k)\\
\Delta x(k)   &= x(k)   - x(k-1)\\
\Delta u(k)   &= u(k)   - u(k-1)\\
\end{align*}
$$

Then, the state-space model is defined as:
$$\Delta x(k+1) = A_d\Delta x(k) + B_d\Delta u(k)$$

Our new state variable vector is:
$$x_n(k) = \begin{bmatrix}\Delta x(k)\\y(k)\end{bmatrix}$$
Note that $y(k+1) - y(k) = C_dA_d\Delta x(k) + C_dB_d\Delta u(k)$

Leading to the following state-space model:
$$
\begin{align}
\underbrace{\begin{bmatrix} \Delta x(k+1) \\ y(k+1)\end{bmatrix}}_{x_n(k+1)} &= \underbrace{\begin{bmatrix} A_d & 0 \\ C_dA_d & 1\end{bmatrix}}_{A_e}\underbrace{\begin{bmatrix} \Delta x(k)\\y(k)\end{bmatrix}}_{x_n(k)} + \underbrace{\begin{bmatrix} B_d\\C_dB_d \end{bmatrix}}_{B_e}\Delta u(k)\\
y(k) &= \underbrace{\begin{bmatrix} 0 & 1\end{bmatrix}}_{C_e}\underbrace{\begin{bmatrix} \Delta x(k)\\y(k) \end{bmatrix}}_{x_n(k)}
\end{align}
$$

In order to implement a MPC approach, we need to have a prediction of the states. In this implementation, the following definition is used to define the state prediction and output variables (Y is N dimensional -> N predictions)
$$
\begin{align}
Y &= \begin{bmatrix} y_0 & y_1 & \dots & y_{N-1}\end{bmatrix}\\
X_n &= \begin{bmatrix} x_{n,0} & x_{n,1} & \dots & x_{n,N-1}\end{bmatrix}\\
\Delta U(k) &= \begin{bmatrix} \Delta u_0 & \Delta u_1 & \dots & \Delta u_{N-1}\end{bmatrix}\\
\end{align}
$$

Leading to:
$$
\begin{align}
Y &= \Phi X_n(k) + \Gamma \Delta U(k)\\
\Phi &= \begin{bmatrix} C_eA_e & C_eA_e^2 & C_eA_e^3 & \cdots & C_eA_e^N\end{bmatrix}^T\\
\Gamma &= \begin{bmatrix} 
C_eB_e & 0 & 0 & 0 & \cdots & 0\\
C_eA_eB_e & C_eB_e & 0 & 0 & \cdots & 0\\
C_eA_e^2B_e & C_eA_eB_e & C_eB_e & 0 & \cdots & 0\\
 &  & \vdots &  & \\
C_eA_e^{N-1}B_e & C_eA_e^{N-2}B_e & C_eA_e^{N-3}B_e & C_eA_e^{N-4}B_e & \cdots & C_eB_e
\end{bmatrix}
\end{align}
$$

The performance matrices are defined using $P$ for the steady-state reference tracking error ($y_N - r_N$), $Q$ the reference tracking error ($y_k - r_k$), and $R$ the for the input difference penalty ($\Delta U(k)$). Then the following weight matrices are defined:
$$
\begin{align*}
\Omega &= \begin{bmatrix} 
Q & 0 & 0 & \cdots & 0 & 0\\
0 & Q & 0 & \cdots & 0 & 0\\
0 & 0 & Q & \cdots & 0 & 0\\
& & \vdots & & &\\
0 & 0 & 0 & \cdots& Q & 0\\
0 & 0 & 0 & \cdots& 0 & P
\end{bmatrix}_{N: 1 \to N}\\
\Psi &= \begin{bmatrix} 
R & 0 & 0 & \cdots & 0 \\
0 & R & 0 & \cdots & 0 \\
0 & 0 & R & \cdots & 0 \\
& & \vdots & & &\\
0 & 0 & 0 & \cdots& R 
\end{bmatrix}_{N: 0 \to N-1}
\end{align*}
$$

The cost function is defined as:
$$
J(X_n(k), \Delta U_k) = \frac{1}{2}\Delta U(k)G\Delta U(k) + F(\Phi X_n(k) - R(k))\Delta U(k)
$$
where
$$
\begin{align*}
R(k) &= \text{References vector}\\
G(k) &= 2(\Psi + \Gamma^T\Omega\Gamma)\\
F(k) &= 2\Gamma^T\Omega
\end{align*}
$$

In [2]:
class MPCController:
    def __init__(self, env, Q, P, R, N=2):
        self.env = env
        self.N = N  # Prediction horizon
        
        self.Ae = np.vstack((np.hstack((env.ad, np.zeros((2,2)))),np.hstack((env.cd*env.ad, env.cd))))
        self.Be = np.vstack((env.bd,env.cd*env.bd))
        self.Ce = np.hstack((np.zeros((2,2)), np.eye(2)))
        
        # Initialize previous state and input vector
        self.x_old = np.zeros((env.ad.shape[0],1))
        self.u_old = np.zeros((self.Be.shape[1],1))

        # Prediction matrices
        self.n_xn = self.Ae.shape[0]  # Number of states in the augmented system
        self.n_du = self.Be.shape[1]  # Number of inputs in the augmented system
        self.n_y  = self.Ce.shape[0]  # Number of outputs in the augmented system
        self.Phi = np.zeros((self.N*self.n_y, self.n_xn))
        self.Gamma = np.zeros((self.N*self.n_y, self.N*self.n_du))
        for i in range(self.N):
            # Phi
            row_init = self.n_y*i;
            row_end  = row_init + self.n_y;
            self.Phi[row_init:row_end,:] = self.Ce @ np.linalg.matrix_power(self.Ae, i+1)

            # Gamma 
            for j in range(i+1):
                col_init = self.n_du*j;
                col_end  = col_init + self.n_du;
                self.Gamma[row_init:row_end,col_init:col_end] = self.Ce @ np.linalg.matrix_power(self.Ae, i-j) @ self.Be
        
        # Performance matrices
        Q = Q*np.eye(self.n_y)     # y-r (reference tracking error)
        P = P*Q                    # y_N-r_N (steady-state reference tracking error)
        R = R*np.eye(self.n_du)    # Δu (input difference)
        
        # Weight matrices
        self.Omega = np.kron(np.diag(np.hstack((np.ones(self.N-1),0))), Q) + \
                     np.kron(np.diag(np.hstack((np.zeros(self.N-1),1))), P)
        self.Psi   = np.kron(np.eye(self.N), R)

    def cost_function_matrices(self, yref):

        # Cost function matrices
        G = 2*(self.Psi + self.Gamma.T @ self.Omega @ self.Gamma)
        F = 2*self.Gamma.T @ self.Omega

        # Vector of reference values
        R_k = np.kron(np.ones((self.N, 1)), yref)

        # Positive semidefinite quadratic term matrix
        Q = (G+G.T)/2
        # Linear term vector
        q = F @ (self.Phi @ self.xn_old - R_k)

        return Q, q
    
    def restriction_matrices(self, umin, umax, dumin, dumax, ymin, ymax):

        C1 = np.kron(np.ones((self.N,1)), np.eye(self.n_du))
        C2 = np.tril(np.kron(np.ones((self.N,self.N)), np.eye(self.n_du)))
        # Input constraints
        M1 = np.vstack((-C2, 
                         C2))
        N1 = np.vstack(( np.kron(-umin, np.ones((self.N,1))) + C1 @ self.u_old, 
                         np.kron( umax, np.ones((self.N,1))) - C1 @ self.u_old))

        # Delta input constraints
        M2 = np.vstack((-np.eye(self.N*self.n_du), 
                         np.eye(self.N*self.n_du)))
        N2 = np.vstack((-np.kron( dumin, np.ones((self.N,1))), 
                         np.kron( dumax, np.ones((self.N,1)))))

        # Output constraints
        M3 = np.vstack((-self.Gamma, 
                         self.Gamma))
        N3 = np.vstack(( np.kron(-ymin, np.ones((self.N,1))) + self.Phi @ self.xn_old, 
                         np.kron( ymax, np.ones((self.N,1))) - self.Phi @ self.xn_old))
        
        M = np.vstack((M1, M2, M3))
        N = np.vstack((N1, N2, N3))

        return M, N
    
    def compute_input(self, x, y, yref, umax=600, dumax=1200, ymax=1.0000e+20):
        # Transform to column vectors
        x = np.array([x]).T
        y = np.array([y]).T
        yref = np.array([yref]).T

        # Define xn_old
        self.xn_old = np.vstack((x - self.x_old,    # measured states - previous states
                                  y))               # measured output
        self.x_old = x;

        # Input constraints
        umax  = np.array([[ umax], 
                          [ umax]])
        umin  = -umax
        # Input variation constraints
        dumax = np.array([[ dumax], 
                          [ dumax]])
        dumin = -dumax
        # Output constraints
        ymax = np.array([[ ymax], 
                         [ ymax]])
        ymin  = -ymax

        # Weights
        Q, q = self.cost_function_matrices(yref)
        M, n = self.restriction_matrices(umin, umax, dumin, dumax, ymin, ymax)

        du = solve_qp(Q,q,M,n, solver="osqp")
        # du = solve_qp(Q,q, solver="osqp")
        u  = np.array([du[0:2]]).T + self.u_old;
        self.u_old = u;

        return u.flatten()